In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# === CONFIG ===
FOLDER = "stationaryClustered" 
# INPUT_CSV = os.path.join("data", FOLDER, "csv", "detailed_summary_report_10.csv")
# BASE_OUTPUT_DIR = os.path.join("data", FOLDER, "charts_10")

INPUT_CSV = os.path.join("data", FOLDER, "combine", "MEAN-combine_summary_report.csv")
#INPUT_CSV = os.path.join("data", FOLDER, "combine", "FIRST-combine_summary_report.csv")
BASE_OUTPUT_DIR = os.path.join("data", FOLDER, "combine-Mean")

ALPHA_PLOT_DIR = os.path.join(BASE_OUTPUT_DIR, "alpha_analysis")
COMPARISON_DIR = os.path.join(BASE_OUTPUT_DIR, "compare_best_alpha")
os.makedirs(ALPHA_PLOT_DIR, exist_ok=True)
os.makedirs(COMPARISON_DIR, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv(INPUT_CSV)
df["LevyFlight.alpha"] = pd.to_numeric(df["LevyFlight.alpha"], errors="coerce")

# Fill missing coverage with 0 for consistency
for col in ["avg_coverage", "min_coverage", "max_coverage"]:
    df[col] = df[col].fillna(0)

# === STEP 1: Analyze alpha for LevyFlight ===
levy_df = df[df["Agent_movementModel"] == "LevyFlight"].copy()
alpha_group = levy_df.groupby("LevyFlight.alpha")["avg_coverage"].mean()
best_alpha = alpha_group.idxmax()
print(f"\n✅ Best alpha for LevyFlight: {best_alpha}")

# === STEP 2: Plot LevyFlight alpha comparison as line chart ===
levy_df = df[df["Agent_movementModel"] == "LevyFlight"].copy()

# Group by alpha and calculate mean coverage stats
alpha_stats = levy_df.groupby("LevyFlight.alpha")[["min_coverage", "avg_coverage", "max_coverage"]].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=alpha_stats, x="LevyFlight.alpha", y="min_coverage", marker="o", label="min_coverage")
sns.lineplot(data=alpha_stats, x="LevyFlight.alpha", y="avg_coverage", marker="o", label="avg_coverage")
sns.lineplot(data=alpha_stats, x="LevyFlight.alpha", y="max_coverage", marker="o", label="max_coverage")

plt.title("Min, Avg, Max Coverage % vs Alpha (LevyFlight)")
plt.xlabel("LevyFlight α")
plt.ylabel("Coverage (%)")
plt.legend(title="Coverage Type")
plt.tight_layout()

out_path = os.path.join(ALPHA_PLOT_DIR, "levy_alpha_comparison_line.png")
plt.savefig(out_path)
plt.close()

print(f"✅ Saved LevyFlight alpha comparison line chart: {out_path}")


# === STEP 3: Filter data to only keep best alpha LevyFlight for comparisons ===
df_filtered = df.copy()
df_filtered = df_filtered[
    (df_filtered["Agent_movementModel"] != "LevyFlight") |
    ((df_filtered["Agent_movementModel"] == "LevyFlight") & (df_filtered["LevyFlight.alpha"] == best_alpha))
]

# === PLOT SETTINGS ===
sns.set(style="whitegrid")
palette = "Set2"

# === Plot 1: Min / Avg / Max Coverage % (split by target)
def plot_coverage_stats(df):
    df_melt = df.melt(
        id_vars=["Agent_movementModel", "Target_movementModel"],
        value_vars=["min_coverage", "avg_coverage", "max_coverage"],
        var_name="Coverage Type",
        value_name="Coverage (%)"
    )
    for target_model, group in df_melt.groupby("Target_movementModel"):
        output_dir = os.path.join(COMPARISON_DIR, str(target_model))
        os.makedirs(output_dir, exist_ok=True)

        plt.figure(figsize=(12, 6))
        sns.pointplot(
            data=group,
            x="Agent_movementModel",
            y="Coverage (%)",
            hue="Coverage Type",
            dodge=True,
            palette=palette
        )
        plt.title(f"Min, Avg, Max Coverage % (Target: {target_model}, Best α = {best_alpha})")
        plt.xticks(rotation=45)
        plt.tight_layout()
        out_path = os.path.join(output_dir, f"coverage_min_avg_max_{target_model}.png")
        plt.savefig(out_path)
        plt.close()
        print(f"✅ Saved: {out_path}")

# === Plot 2–5: Time to reach coverage threshold

# === NEW: Combined time-to-coverage line chart ===
def plot_combined_time_chart(df):
    # Prepare data for plotting
    time_cols = [f"time_to_{pct}_percent" for pct in [25, 50, 75, 90]]
    df_melt = df.melt(
        id_vars=["Agent_movementModel", "Target_movementModel"],
        value_vars=time_cols,
        var_name="Coverage Threshold",
        value_name="Time (seconds)"
    )

    # Convert "time_to_X_percent" -> integer percentage
    df_melt["Coverage (%)"] = df_melt["Coverage Threshold"].str.extract(r"(\d+)").astype(int)

    # Plot per target model
    for target_model, group in df_melt.groupby("Target_movementModel"):
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=group,
            x="Coverage (%)",
            y="Time (seconds)",
            hue="Agent_movementModel",
            marker="o"
        )
        plt.title(f"Time to Coverage — Target: {target_model} (Best α = {best_alpha})")
        plt.xlabel("Coverage (%)")
        plt.ylabel("Time (seconds)")
        plt.xticks([25, 50, 75, 90])
        plt.tight_layout()

        out_path = os.path.join(COMPARISON_DIR, str(target_model), f"combined_time_to_coverage_{target_model}.png")
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        plt.savefig(out_path)
        plt.close()
        print(f"✅ Saved combined chart: {out_path}")


# # === GENERATE FINAL CHARTS ===
# plot_coverage_stats(df_filtered)
# for pct in [25, 50, 75, 90]:
#     plot_time_to_coverage(df_filtered, pct)

# === GENERATE FINAL CHARTS ===
plot_coverage_stats(df_filtered)  # Keep your coverage chart
plot_combined_time_chart(df_filtered)  # New combined time-to-coverage chart

print(f"\n🎉 All plots and alpha analysis saved under: {BASE_OUTPUT_DIR}")



✅ Best alpha for LevyFlight: 0.5
✅ Saved LevyFlight alpha comparison line chart: data\stationaryClustered\combine-Mean\alpha_analysis\levy_alpha_comparison_line.png
✅ Saved: data\stationaryClustered\combine-Mean\compare_best_alpha\StationaryClustered\coverage_min_avg_max_StationaryClustered.png
✅ Saved combined chart: data\stationaryClustered\combine-Mean\compare_best_alpha\StationaryClustered\combined_time_to_coverage_StationaryClustered.png

🎉 All plots and alpha analysis saved under: data\stationaryClustered\combine-Mean
